<a href="https://colab.research.google.com/github/carsofferrei/04_data_processing/blob/main/spark_streaming/challenges/final_challenges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up PySpark

In [1]:
%pip install pyspark

# Context
Message events are coming from platform message broker (kafka, pubsub, kinesis...).
You need to process the data according to the requirements.

Message schema:
- timestamp
- value
- event_type
- message_id
- country_id
- user_id



# Challenge 1

Step 1
- Change exising producer
	- Change parquet location to "/content/lake/bronze/messages/data"
	- Add checkpoint (/content/lake/bronze/messages/checkpoint)
	- Delete /content/lake/bronze/messages and reprocess data
	- For reprocessing, run the streaming for at least 1 minute, then stop it

Step 2
- Implement new stream job to read from messages in bronze layer and split result in two locations
	- "messages_corrupted"
		- logic: event_status is null, empty or equal to "NONE"
    - extra logic: add country name by joining message with countries dataset
		- partition by "date" -extract it from timestamp
		- location: /content/lake/silver/messages_corrupted/data

	- "messages"
		- logic: not corrupted data
		- extra logic: add country name by joining message with countries dataset
		- partition by "date" -extract it from timestamp
		- location: /content/lake/silver/messages/data

	- technical requirements
		- add checkpint (choose location)
		- use StructSchema
		- Set trigger interval to 5 seconds
		- run streaming for at least 20 seconds, then stop it

	- alternatives
		- implementing single streaming job with foreach/- foreachBatch logic to write into two locations
		- implementing two streaming jobs, one for messages and another for messages_corrupted
		- (paying attention on the paths and checkpoints)


  - Check results:
    - results from messages in bronze layer should match with the sum of messages+messages_corrupted in the silver layer

In [3]:
%pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 43.9 MB/s eta 0:00:00


# Producer

In [34]:
import pyspark.sql.functions as F
from pyspark.sql import DataFrame
from faker import Faker
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Test streaming').getOrCreate()
sc = spark.sparkContext

fake = Faker()
messages = [fake.uuid4() for _ in range(50)]

def enrich_data(df, messages=messages):
  fake = Faker()
  new_columns = {
      'event_type': F.lit(fake.random_element(elements=('OPEN', 'RECEIVED', 'SENT', 'CREATED', 'CLICKED', '', 'NONE'))),
      'message_id': F.lit(fake.random_element(elements=messages)),
      'channel': F.lit(fake.random_element(elements=('CHAT', 'EMAIL', 'SMS', 'PUSH', 'OTHER'))),
      'country_id': F.lit(fake.random_int(min=2000, max=2015)),
      'user_id': F.lit(fake.random_int(min=1000, max=1050)),
  }
  df = df.withColumns(new_columns)
  return df

def insert_messages(df: DataFrame, batch_id):
  enrich = enrich_data(df)
  enrich.write.mode("append").format("parquet").save("content/lake/bronze/messages/data")

# read stream
df_stream = spark.readStream.format("rate").option("rowsPerSecond", 1).load()

# write stream
query = (df_stream
         .writeStream
         .outputMode('append')
         .option('checkpointLocation', "content/lake/bronze/checkpoint")
         .trigger(processingTime='1 seconds')
         .foreachBatch(insert_messages)
         .start()
)

query.awaitTermination(60)


False

In [17]:
query.isActive

True

In [18]:
query.stop()

In [19]:
df = spark.read.format("parquet").load("content/lake/bronze/messages/data/*")
df.count()

72

# Additional datasets

In [20]:
countries = [
    {"country_id": 2000, "country": "Brazil"},
    {"country_id": 2001, "country": "Portugal"},
    {"country_id": 2002, "country": "Spain"},
    {"country_id": 2003, "country": "Germany"},
    {"country_id": 2004, "country": "France"},
    {"country_id": 2005, "country": "Italy"},
    {"country_id": 2006, "country": "United Kingdom"},
    {"country_id": 2007, "country": "United States"},
    {"country_id": 2008, "country": "Canada"},
    {"country_id": 2009, "country": "Australia"},
    {"country_id": 2010, "country": "Japan"},
    {"country_id": 2011, "country": "China"},
    {"country_id": 2012, "country": "India"},
    {"country_id": 2013, "country": "South Korea"},
    {"country_id": 2014, "country": "Russia"},
    {"country_id": 2015, "country": "Argentina"}
]

countries = spark.createDataFrame(countries)

# Streaming Messages x Messages Corrupted

In [33]:
!rm -rf content/lake/

In [41]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time

def cleansing_silver(df: DataFrame, batch_id):
    if df.isEmpty():
        print("No data in the current batch.")
        return

    # Join with countries
    df = df.join(countries, "country_id", "left")

    print(f'Filtering corrupted messages...')
    corrupted_df = df.filter((col("event_type").isNull() |
                              (col("event_type") == "") |
                              (col("event_type") == "NONE")
                              ))

    print(f'Saving corrupted messages...')
    corrupted_df.write.mode("append").format("parquet").partitionBy("date").save("content/lake/silver/messages_corrupted/data")

    print(f'Creating dataframe without corrupted messages...')
    clean_df = df.filter(~(col("event_type").isNull() |
                          (col("event_type") == "") |
                          (col("event_type") == "NONE"))
                        )
    print(f'Saving clean messages...')
    clean_df.write.mode("append").format("parquet").partitionBy("date").save("content/lake/silver/messages/data")


print(f'Define streaming schema...')
clean_schema = StructType([
    StructField('timestamp', TimestampType(), True),
    StructField('date', DateType(), True),
    StructField('user_id', IntegerType(), True),
    StructField('value', IntegerType(), True),
    StructField('event_type', StringType(), True),
    StructField('channel', StringType(), True),
    StructField('message_id', StringType(), True),
    StructField('country_id', IntegerType(), True),
    StructField('country', StringType(), True),
])

print(f'Read the streaming data...')
cleansing_silver_data = spark.readStream.format("parquet").schema(clean_schema).load("content/lake/bronze/messages/data/*")

print(f'Create a new date column that will be the split column in writeStreaming...')
cleansing_silver_data = cleansing_silver_data.withColumn("date", F.to_date("timestamp"))

print(f'Write Streaming...')
cleansing_silver_query = (cleansing_silver_data
                          .writeStream
                          .outputMode('append')
                          .option('checkpointLocation', 'content/lake/silver/checkpoint')
                          .trigger(processingTime='5 seconds')
                          .foreachBatch(cleansing_silver)
                          .start()
                          )

query.awaitTermination(20)

Define streaming schema...
Read the streaming data...


ERROR:py4j.clientserver:There was an exception while executing the Python Proxy on the Python Side.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 617, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/sql/utils.py", line 120, in call
    raise e
  File "/usr/local/lib/python3.10/dist-packages/pyspark/sql/utils.py", line 117, in call
    self.func(DataFrame(jdf, wrapped_session_jdf), batch_id)
  File "<ipython-input-41-3ea66b45bbec>", line 6, in cleansing_silver
    if df.isEmpty():
  File "/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py", line 885, in isEmpty
    return self._jdf.isEmpty()
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1322, in __call__
    return_value = get_return_value(
  File "/usr/local/lib/python3.10/dist-packages/pyspark/errors/exceptions/captured.py", line 179, in deco


False

In [26]:
cleansing_silver_query.isActive

False

## Checking data

In [28]:
def checking_silver_data():
    bronze_count = spark.read.format("parquet").load("content/lake/bronze/messages/data/*").count()
    clean_count = spark.read.format("parquet").load("content/lake/silver/messages/data/*").count()
    corrupted_count = spark.read.format("parquet").load("content/lake/silver/messages_corrupted/data/*").count()

    assert bronze_count == clean_count + corrupted_count, "Dataframes doesn't matches. Did you run the code more than once? Be careful with append mode"
    print(f'Validation passed: Bronze [{bronze_count}] = Clean [{clean_count}] + Corrupted [{corrupted_count}]')

checking_silver_data()

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/content/content/lake/silver/messages/data/*.

# Challenge 2

- Run business report
- But first, there is a bug in the system which is causing some duplicated messages, we need to exclude these lines from the report

- removing duplicates logic:
  - Identify possible duplicates on message_id, event_type and channel
  - in case of duplicates, consider only the first message (occurrence by timestamp)
  - Ex:
    In table below, the correct message to consider is the second line

```
    message_id | channel | event_type | timestamp
    123        | CHAT    | CREATED    | 10:10:01
    123        | CHAT    | CREATED    | 07:56:45 (first occurrence)
    123        | CHAT    | CREATED    | 08:13:33
```

- After cleaning the data we're able to create the busines report

In [57]:
# dedup data
from pyspark.sql import functions as F
from pyspark.sql.window import Window
df = spark.read.format("parquet").load("/content/lake/silver/messages")
dedup = df.withColumn("row_number", F.row_number().over(Window.partitionBy("message_id", "event_type", "channel").orderBy("timestamp"))).filter("row_number = 1").drop("row_number")

In [68]:
dedup.limit(3).show()

+----------+--------------------+-----+----------+--------------------+-------+-------+--------+----------+
|country_id|           timestamp|value|event_type|          message_id|channel|user_id| country|      date|
+----------+--------------------+-----+----------+--------------------+-------+-------+--------+----------+
|      2001|2024-12-10 22:20:...|   10|  RECEIVED|02ad4c7c-8fd1-430...|    SMS|   1042|Portugal|2024-12-10|
|      2002|2024-12-10 22:21:...|  125|      OPEN|073aa43d-3433-4da...|   CHAT|   1023|   Spain|2024-12-10|
|      2011|2024-12-10 22:19:...|    5|   CLICKED|0cb04e29-f7e9-402...|  OTHER|   1020|   China|2024-12-10|
+----------+--------------------+-----+----------+--------------------+-------+-------+--------+----------+



### Report 1
  - Aggregate data by date, event_type and channel
  - Count number of messages
  - pivot event_type from rows into columns
  - schema expected:
  
```
|      date|channel|CLICKED|CREATED|OPEN|RECEIVED|SENT|
+----------+-------+-------+-------+----+--------+----+
|2024-12-03|    SMS|      4|      4|   1|       1|   5|
|2024-12-03|   CHAT|      3|      7|   5|       8|   4|
|2024-12-03|   PUSH|   NULL|      3|   4|       3|   4|
```

In [81]:
print(f'Obtaining the pivot and expected schema')
df.groupBy("date", "channel").pivot("event_type").agg(count("*").alias("event_count")).fillna(0).show()

Obtaining the pivot and expected schema
+----------+-------+-------+-------+----+--------+----+
|      date|channel|CLICKED|CREATED|OPEN|RECEIVED|SENT|
+----------+-------+-------+-------+----+--------+----+
|2024-12-10|  OTHER|      6|      2|   2|       6|   5|
|2024-12-10|  EMAIL|      2|      5|   1|       1|   3|
|2024-12-10|    SMS|      1|      3|   3|       4|   3|
|2024-12-10|   CHAT|      2|      6|   4|       5|   3|
|2024-12-10|   PUSH|      3|      4|   2|       4|   6|
+----------+-------+-------+-------+----+--------+----+



## Report 2

- Identify the most active users by channel (sorted by number of iterations)
- schema expected:

```
+-------+----------+----+-----+-----+----+---+
|user_id|iterations|CHAT|EMAIL|OTHER|PUSH|SMS|
+-------+----------+----+-----+-----+----+---+
|   1022|         5|   2|    0|    1|   0|  2|
|   1004|         4|   1|    1|    1|   1|  0|
|   1013|         4|   0|    0|    2|   1|  1|
|   1020|         4|   2|    0|    1|   1|  0|
```


In [80]:
from pyspark.sql.functions import col, expr, lit

print(f'Calculating number of interactions per channel first')
agg_channel_user = df.groupBy("user_id").pivot("channel").agg(count("*").alias("iterations")).fillna(0)

print(f'Aggregating by user_id, the sum of all channels')
columns_to_sum = [c for c in agg_channel_user.columns if c not in ["user_id"]]
sum_expression = " + ".join([f"`{col_name}`" for col_name in columns_to_sum])

print(f'Creating a column with the total number of iterations, resulting from the sum of interactions for each channel')
agg_df = agg_channel_user.withColumn("iterations", expr(sum_expression))

print(f'Applying expected schema...')
agg_df_final = agg_df.select(
                            col("user_id"),
                            col("iterations"),
                            *[col(c) for c in columns_to_sum]
                        )

print(f'Sorting the result to obtain the most active users...')
agg_df_final.sort(desc("iterations")).limit(4).show()

Calculating number of interactions per channel first
Aggregating by user_id, the sum of all channels
Creating a column with the total number of iterations, resulting from the sum of interactions for each channel
Applying expected schema...
Sorting the result to obtain the most active users...
+-------+----------+----+-----+-----+----+---+
|user_id|iterations|CHAT|EMAIL|OTHER|PUSH|SMS|
+-------+----------+----+-----+-----+----+---+
|   1019|         6|   1|    1|    1|   1|  2|
|   1037|         5|   2|    1|    2|   0|  0|
|   1006|         5|   1|    0|    2|   2|  0|
|   1010|         4|   0|    0|    3|   1|  0|
+-------+----------+----+-----+-----+----+---+



# Challenge 3

In [ ]:
# Theoretical question:

# A new usecase requires the message data to be aggregate in near real time
# They want to build a dashboard embedded in the platform website to analyze message data in low latency (few minutes)
# This application will access directly the data aggregated by streaming process

# Q1:
- What would be your suggestion to achieve that using Spark Structure Streaming?
Or would you choose a different data processing tool?

- Which storage would you use and why? (database?, data lake?, kafka?)

